In [3]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine pedirá um token de autorização.
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para que possamos salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
Dependências instaladas.
Autenticando na API do Google Earth Engine...
API do Google Earth Engine inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [4]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [5]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2024-01-01'

# 2. Área de Interesse(AOI)
# Coleção de biomas do Brasil carregada e filtrada para obter apenas o polígono do Cerrado.
# Este polígono 'cerrado_aoi' será usado para "recortar" todos os outros dados.
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2024-01-01
Área de Interesse: Bioma Cerrado


In [6]:
# ==============================================================================
# PASSO 2: COLETA DA VARIÁVEL ALVO (ÁREA QUEIMADA)
# ==============================================================================

# 1. Carregamos o asset global como uma única ee.Image, SEM NENHUM CORTE.
burned_area_image_global = ee.Image('projects/mapbiomas-public/assets/brazil/fire/collection4/mapbiomas_fire_collection4_monthly_burned_v1')

# --- TESTE DE VERIFICAÇÃO SEGURO ---
# Agora fazemos as perguntas ao objeto global e simples.
print("Iniciando verificação segura no objeto global...")

# Nomes de todas as bandas do objeto global.
band_names_server = burned_area_image_global.bandNames()

# Pedimos ao servidor para nos enviar o NÚMERO TOTAL de bandas.
num_bands = band_names_server.size().getInfo()
print(f"O objeto global é uma imagem única com {num_bands} bandas.")

# Pedimos ao servidor para "fatiar" a lista e nos enviar APENAS OS 5 PRIMEIROS NOMES.
first_5_bands = band_names_server.slice(35, 40).getInfo()
print("Exemplo dos primeiros 5 nomes de banda:", first_5_bands)
print("Verificação concluída com sucesso.")

# ------------------------------------------------------------------------------

# 2. Objeto cortado para uso futuro
burned_area_cerrado = burned_area_image_global.clip(cerrado_aoi)

print("\nObjeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.")

Iniciando verificação segura no objeto global...
O objeto global é uma imagem única com 40 bandas.
Exemplo dos primeiros 5 nomes de banda: ['burned_monthly_2020', 'burned_monthly_2021', 'burned_monthly_2022', 'burned_monthly_2023', 'burned_monthly_2024']
Verificação concluída com sucesso.

Objeto 'burned_area_cerrado' (recortado para o Cerrado) criado para uso futuro.


In [7]:
# ==============================================================================
# PASSO 3: COLETA DAS VARIÁVEIS PREDITIVAS (FEATURES)
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi)

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').clip(cerrado_aoi)
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- 3.4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2').clip(cerrado_aoi)

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87648 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3631 imagens de LST (diárias).
Dados de elevação e declive carregados.
Dados de uso e cobertura do solo carregados.


In [10]:
# ==============================================================================
# PASSO 2.1.1 : COLETA E TRANSFORMAÇÃO DOS DADOS DE FOCOS DE CALOR
# ==============================================================================

# 1. Carregamos a coleção de IMAGENS diárias de fogo do MODIS (Produto MOD14A1).
# Cada imagem tem uma banda 'FireMask' que classifica os pixels.
daily_fire_images = ee.ImageCollection('MODIS/061/MOD14A1') \
   .filterDate(START_DATE, END_DATE) \
   .filterBounds(cerrado_aoi) \
   .select('FireMask')

# 2. Função para transformar os pixels de fogo de uma imagem diária em pontos (features).
def raster_to_points(image):
    # A data da imagem é a informação mais importante. Vamos extraí-la.
    # E vamos adicioná-la como uma propriedade em cada ponto que gerarmos.
    date = image.date().format('YYYY-MM-dd')

    # Criamos uma máscara, mantendo apenas os pixels classificados como fogo (valores >= 7).
    fire_pixels = image.gte(7)

    # Para evitar processar áreas vazias, usamos a máscara para "apagar" os pixels sem fogo.
    fire_pixels = fire_pixels.selfMask()

    # Convertemos os pixels de fogo restantes em vetores (pontos).
    # O GEE cria um ponto no centro de cada pixel de fogo.
    fire_points = fire_pixels.reduceToVectors(
        geometry=image.geometry(), # Processa a geometria da imagem inteira
        scale=1000,                # A resolução nativa do MODIS é ~1km
        geometryType='centroid',
        eightConnected=False
    )

    # Se nenhum ponto de fogo for encontrado na imagem, retorna uma coleção vazia.
    # Senão, adiciona a data a cada ponto encontrado.
    return ee.Algorithms.If(
        fire_points.size(),
        fire_points.map(lambda point: point.set('ACQ_DATE', date)),
        ee.FeatureCollection()
    )

# 3. Aplicamos a função a cada imagem da coleção e depois "achatamos" o resultado.
# O.map() cria uma coleção de coleções. O.flatten() junta tudo em uma única
# e gigantesca coleção de todos os pontos de fogo de todos os dias.
active_fires_collection = daily_fire_images.map(raster_to_points).flatten()


# --- Verificação Final ---
# Agora temos uma FeatureCollection funcional, exatamente como precisávamos antes.
# Vamos inspecionar o primeiro ponto para confirmar que a estrutura está correta.
print("Coleção de imagens diárias convertida para uma FeatureCollection de pontos de fogo.")
print("\nInspecionando o primeiro ponto de fogo gerado:")
print(active_fires_collection.first().getInfo())

Objeto 'active_fires_collection' criado no servidor.


In [ ]:
# ==============================================================================
# PASSO 2.1.3: ALGORITMO DE GERAÇÃO DE RÓTULOS (EXEMPLO PARA O ANO DE 2022)
# ==============================================================================

# --- 1. Selecionar o ano de interesse ---
YEAR_TO_PROCESS = 2022
band_name = f'burned_monthly_{YEAR_TO_PROCESS}'

# Seleciona a banda do ano de 2022 da nossa imagem de área queimada.
burned_area_2022 = burned_area_cerrado.select(band_name)

# --- 2. Identificar as manchas de incêndio (cicatrizes) ---
# Cria uma máscara binária: 1 para pixels queimados, 0 para não queimados.
# O valor do pixel no MapBiomas Fogo indica o mês (1-12), então qualquer valor > 0 é queimado.
fire_mask = burned_area_2022.gt(0)

# Agrupa os pixels queimados conectados em "manchas" com IDs únicos.
# É como usar a varinha mágica no Photoshop.
fire_patches = fire_mask.connectedComponents(
    connectedness=ee.Kernel.plus(1),
    maxSize=1024
)

# --- 3. Filtrar os focos de calor para o mesmo ano ---
active_fires_2022 = active_fires_collection.filterDate(f'{YEAR_TO_PROCESS}-01-01', f'{YEAR_TO_PROCESS}-12-31')

# --- 4. Encontrar o primeiro foco de calor em cada mancha ---
# O método.reduceRegions() é extremamente poderoso. Ele vai:
# 1. Pegar cada mancha de incêndio (da 'fire_patches').
# 2. Encontrar todos os focos de calor ('active_fires_2022') que caem dentro dela.
# 3. Aplicar um 'reducer' a esse grupo de pontos.

# Nosso reducer vai encontrar o ponto com a menor data de aquisição.
# Para isso, primeiro ordenamos os pontos por data e depois pegamos o primeiro.
first_ignition_reducer = ee.Reducer.first().setOutputs(['ignition'])

# Executa a operação de redução.
# O resultado será uma FeatureCollection onde cada feature é uma mancha de incêndio,
# e suas propriedades conterão as informações do primeiro foco de calor encontrado nela.
ignition_points_raw = active_fires_2022.sort('ACQ_DATE').reduceRegions(
    collection=fire_patches,
    reducer=first_ignition_reducer,
    scale=TARGET_RESOLUTION
)

# --- 5. Limpar e formatar os resultados ---
# A saída de.reduceRegions() pode ser um pouco suja. Vamos limpar.
# Removemos geometrias nulas e selecionamos apenas as propriedades que nos interessam.
def format_ignition_point(feature):
    # Extrai as propriedades do ponto de ignição que o reducer encontrou.
    ignition_properties = ee.Dictionary(feature.get('ignition'))

    # Retorna uma nova feature com a geometria do ponto e as propriedades formatadas.
    return ee.Feature(
        ee.Geometry.Point([ignition_properties.get('longitude'), ignition_properties.get('latitude')]),
        {
            'acq_date': ignition_properties.get('ACQ_DATE'),
            'acq_time': ignition_properties.get('ACQ_TIME'),
            'confidence': ignition_properties.get('confidence'),
            'frp': ignition_properties.get('frp') # Fire Radiative Power, uma medida da intensidade
        }
    )

# Filtra os resultados que não encontraram nenhum foco de calor e aplica a formatação.
ignition_points_2022 = ignition_points_raw.filter(ee.Filter.neq('ignition', None)).map(format_ignition_point)


# --- Verificação Final ---
# Vamos contar quantos pontos de ignição únicos encontramos para 2022.
# Desta vez,.getInfo() é seguro, pois o número de incêndios é muito menor que o número de pixels.
ignition_count = ignition_points_2022.size().getInfo()
print(f"Número de pontos de ignição gerados para o ano de {YEAR_TO_PROCESS}: {ignition_count}")

# Vamos inspecionar o primeiro ponto para ver como ficou.
print("\nExemplo do primeiro ponto de ignição encontrado:")
print(ignition_points_2022.first().getInfo())

In [ ]:
# ==============================================================================
# PASSO 2.1.4: EXPORTAR O DATASET DE RÓTULOS PARA O GOOGLE DRIVE
# ==============================================================================

# Define o nome do arquivo e a tarefa de exportação.
output_filename = f'ignition_points_cerrado_{YEAR_TO_PROCESS}'
task = ee.batch.Export.table.toDrive(
    collection=ignition_points_2022,
    description=output_filename,
    folder='Projeto_Incendios/data/raw',  # A pasta que criamos no seu Google Drive
    fileNamePrefix=output_filename,
    fileFormat='CSV'
)

# Inicia a tarefa de exportação.
task.start()

print("Tarefa de exportação iniciada. Monitore o status na aba 'Tasks' do Google Earth Engine.")
print(f"O arquivo '{output_filename}.csv' será salvo na pasta 'Projeto_Incendios/data/raw' do seu Google Drive.")